In [1]:
import cv2
import numpy as np
import imutils

TURMERIC_MIN_ASPECT_RATIO = 0.2  
TURMERIC_MAX_ASPECT_RATIO = 5.0
TURMERIC_MIN_AREA = 1000
AFFECTED_AREA_THRESHOLD = 0.2

def resize_image(image, width, height):
    """Resize image for display purposes."""
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def detect_turmeric_quality(image_path):
    """Detect turmeric quality, calculate dimensions, and grade the results."""
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Input image not found or failed to load.")
        return
    print("Input image loaded successfully.")

    # Display original image
    cv2.imshow("Original Image", resize_image(image, 800, 600))

    alpha = 2.0
    beta = 0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    cv2.imshow("Brightness and Contrast Adjusted", resize_image(adjusted_image, 800, 600))

    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)

    lower_turmeric = np.array([20, 50, 50])
    upper_turmeric = np.array([35, 255, 255]) 
    turmeric_mask = cv2.inRange(hsv, lower_turmeric, upper_turmeric)
    cv2.imshow("Turmeric Mask", resize_image(turmeric_mask, 800, 600))

    kernel = np.ones((5, 5), np.uint8)
    turmeric_mask = cv2.morphologyEx(turmeric_mask, cv2.MORPH_CLOSE, kernel)
    turmeric_mask = cv2.morphologyEx(turmeric_mask, cv2.MORPH_OPEN, kernel)
    cv2.imshow("Morphological Cleaned Mask", resize_image(turmeric_mask, 800, 600))

    cnts = cv2.findContours(turmeric_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = adjusted_image.copy()

    lower_affected = np.array([0, 50, 50])
    upper_affected = np.array([20, 255, 255])
    mask_affected = cv2.inRange(hsv, lower_affected, upper_affected)
    cv2.imshow("Affected Area Mask", resize_image(mask_affected, 800, 600))

    # Grading counters
    total_turmeric = 0
    bad_quality_turmeric = 0

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > TURMERIC_MIN_AREA:
            x, y, w, h = cv2.boundingRect(c)
            aspect_ratio = float(h) / w if w > 0 else 0

            if TURMERIC_MIN_ASPECT_RATIO <= aspect_ratio <= TURMERIC_MAX_ASPECT_RATIO:
                total_turmeric += 1

                turmeric_length = h
                turmeric_width = w

                single_turmeric_mask = np.zeros(turmeric_mask.shape, dtype=np.uint8)
                cv2.drawContours(single_turmeric_mask, [c], -1, 255, thickness=cv2.FILLED)

                # Isolate affected areas within the turmeric mask region
                affected_turmeric = cv2.bitwise_and(mask_affected, mask_affected, mask=single_turmeric_mask)

                cnts_affected = cv2.findContours(affected_turmeric, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts_affected = imutils.grab_contours(cnts_affected)

                for affected_contour in cnts_affected:
                    cv2.drawContours(output_image, [affected_contour], -1, (0, 255, 255), 2)  # Yellow for affected regions

                affected_area = cv2.countNonZero(affected_turmeric)
                affected_ratio = affected_area / area if area > 0 else 0

                if affected_ratio > AFFECTED_AREA_THRESHOLD:
                    bad_quality_turmeric += 1
                    label = "Bad Quality Turmeric"
                    color = (0, 0, 255)
                else:
                    label = "Good Quality Turmeric"
                    color = (0, 255, 0)

                cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                cv2.putText(output_image, f"Length: {round(turmeric_length * 0.0264583333)}cm", 
                            (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                cv2.putText(output_image, f"Width: {round(turmeric_width * 0.0264583333)}cm", 
                            (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    print(f"Total Turmeric Pieces: {total_turmeric}")
    print(f"Bad Quality Turmeric: {bad_quality_turmeric}")
    print(f"Good Quality Turmeric: {total_turmeric - bad_quality_turmeric}")

    grading_text = f"Total: {total_turmeric}, Bad: {bad_quality_turmeric}, Good: {total_turmeric - bad_quality_turmeric}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

    # Display final output
    cv2.imshow("Final Output", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the input image
image_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\clove\20250124_145051.jpg'
detect_turmeric_quality(image_path)


Error: Input image not found or failed to load.
